In [6]:
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import ascii

#### Miscellaneous functions

In [7]:
def flatten(array): # when np.flatten() doesn't work
    temp = []
    for sublist in array:
        for item in sublist:
            temp.append(item)
    return np.array(temp)
  
def rmv(array,out): # removing galaxies in "out" from galaxies in "array"
    test_out =[] 
    for i in array:
        if i in out:
            test_out.append(False)
        else:
            test_out.append(True)
           
    return np.array(array)[test_out]


### V^2 classification

In [8]:
galzones = Table.read('DR7_galzones.dat', format = 'ascii') 
zonevoids = Table.read('DR7_zonevoids.dat', format = 'ascii')
total_glx = len(galzones)

#### edge and outside galaxies

In [9]:
edge_bool = [bool(x) for x in np.array(np.array(galzones['edge']))] 
edge_gal = np.array(galzones['gal'][edge_bool])

In [10]:
out_bool = [bool(x) for x in np.array(np.array(galzones['out']))]
out_gal = np.array(galzones['gal'][out_bool])

#### void galaxies

In [11]:
void_zones = zonevoids[(zonevoids['void0'] != -1)]['zone'] # zones that are in voids

In [12]:
void_gal_ = []
for i in void_zones:
    void_gal_.append(list(galzones['gal'][galzones['zone'] == i]))

void_gal_flat = flatten(void_gal_) # around 200,000 are void galaxies

#### no outside galaxies within voids but  there are edge galaxies within voids

In [13]:
void_gal = rmv(void_gal_flat,edge_gal) # eliminating edge galazies, True for non edge, about 8000

#### wall galaxies

In [14]:
non_void_zones = zonevoids[(zonevoids['void0'] == -1)]['zone'] # zones that are not in voids

non_void_gal = []
for i in non_void_zones:
    non_void_gal.append(list(galzones['gal'][galzones['zone'] == i]))
        
wall_gal = rmv(flatten(non_void_gal),edge_gal)

### Check

In [15]:
len(wall_gal)+len(void_gal)+len(edge_gal)+len(out_gal) == total_glx

True

### Voidfinder classification

In [16]:
data_voidfinder = Table.read('ALL_vflag_comoving.txt', format = 'ascii')

In [17]:
wall_gal_vf = np.argwhere(np.array(data_voidfinder['vflag']) == 0)
void_gal_vf = np.argwhere(np.array(data_voidfinder['vflag']) == 1)

In [18]:
out_gal_vf = np.argwhere(np.array(data_voidfinder['vflag']) == 9)
edge_gal_vf = np.argwhere(np.array(data_voidfinder['vflag']) == 2)

### Check

In [19]:
len(out_gal_vf)+len(edge_gal_vf)+len(wall_gal_vf)+len(void_gal_vf) == len(data_voidfinder)

True

### add flag columns and save file

In [ ]:
flag_vsquared=[] # checks all galaxies in each statement as a double check for any overlaps
for i in range(len(data)):
    if i in wall_gal:
        flag_vsquared.append(0)
    if i in void_gal:
        flag_vsquared.append(1)
    if i in edge_gal:
        flag_vsquared.append(2)
    if i in out_gal:
        flag_vsquared.append(9)
        
flag_vsquared_vf=[]
for i in range(len(data)):
    if i in wall_gal_vf:
        flag_vsquared_vf.append(0)
    if i in void_gal_vf:
        flag_vsquared_vf.append(1)
    if i in edge_gal_vf:
        flag_vsquared_vf.append(2)
    if i in out_gal_vf:
        flag_vsquared_vf.append(9)
        
print(len(flag_vsquared),len(flag_vsquared_vf), len(data)) #check



In [ ]:
flags_table = Table([flag_vsquared_vf,flag_vsquared], names=('flag_voidfinder', 'flag_vsquared'))

data.add_columns(flags_table.columns.values())

ascii.write(data, 'data_flags.dat')  